# 04. Data Processing

### Importando as bibliotecas

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import params.consts as consts

### Lendo a base de dados

In [2]:
df = pd.read_csv(consts.DATABASE) # Armazenando a base de dados em uma variável

df.head(3) # Exibindo as 3 primeiras linhas da base de dados

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0


### Excluindo as colunas que contém valores únicos
- **ID:** Excluindo a coluna ID para deixar o dataset apenas com informações relevantes, já que uma coluna com identificadores únicos para cada cliente não acrescenta em nada nesse estudo.

In [3]:
df = df.drop('ID', axis=1) # Excluindo as colunas com valores únicos

### Excluindo as colunas que contém valores constantes
- **Z_CostContact:** Excluindo a coluna Z_CostContact para deixar o dataset apenas com informações relevantes, já que uma coluna com valores únicos não acrescenta em nada nesse estudo.
- **Z_Revenue:** Excluindo a coluna Z_Revenue para deixar o dataset apenas com informações relevantes, já que uma coluna com valores únicos não acrescenta em nada nesse estudo.

In [4]:
df = df.drop(['Z_CostContact', 'Z_Revenue'], axis=1) # Excluindo as colunas com valores constantes

### Corrigindo os tipos de dados das colunas
- **Dt_Customer:** Convertendo essa coluna para o formato de data.

In [5]:
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], format="%Y-%m-%d") # Convertendo a coluna Dt_Customer para o formato de data

### Tratando as colunas com valores nulos
- **Income:** Como essa coluna possui apenas 24 valores nulos em um univeso de 2.240 (o que representa cerca de 1% da base), optamos por remover essas linhas para seguir o projeto, ao invés de substituir esses valores pela média ou pela mediana, que não é tão sensível a outliers como a média.

In [6]:
df = df.dropna(subset=['Income']) # Removendo as linhas com valores nulos na coluna Income

### Tratando as colunas com valores outliers

### Criando colunas de tempo/idade
- **Dt_Customer e Days_Since_Enrolled:** Criando a coluna Days_Since_Enrolled para usar a data da coluna Dt_Customer de forma mais apropriada para os modelos, onde a informação de tempo/idade é mais relevante que uma data.
- **Dt_Customer e Years_Since_Enrolled:** Criando a coluna Years_Since_Enrolled para usar a data da coluna Dt_Customer de forma mais apropriada para os modelos, onde a informação de tempo/idade é mais relevante que uma data.
- **Year_Birth e Age:** Criando a coluna Age para usar a data da coluna Year_Birth de forma mais apropriada para os modelos, onde a informação de tempo/idade é mais relevante que uma data.

In [7]:
df['Days_Since_Enrolled'] = df['Dt_Customer'].max() - df['Dt_Customer'] # Criando a coluna Days_Since_Enrolled com a diferença do valor máximo das datas e a data de cadastro do cliente
df['Days_Since_Enrolled'] = df['Days_Since_Enrolled'].dt.days # Convertendo o tempo para dias

df['Years_Since_Enrolled'] = df['Days_Since_Enrolled'] // 365 # Criando a coluna Years_Since_Enrolled apenas com anos inteiros (dividindo com 2 barras)

df['Age'] = df['Dt_Customer'].max().year - df['Year_Birth'] # Criando a coluna Age com a idade dos clientes, fixando o valor máximmo das datas da campanha para que as idades não fiquem variando de acordo com o tempo que o código é rodado

### Unificando / somando colunas semelhantes
- **MntTotal:** Criando a coluna MntTotal com o somatório dos valores gastos em todas as categorias de produtos.
- **MntRegularProds:** Criando a coluna MntRegularProds com o valor gasto somente em produtos regulares.

In [8]:
df['MntTotal'] = ( # Criando a coluna com o somatório das colunas de todas as categorias de produtos
    df['MntWines'] + 
    df['MntFruits'] + 
    df['MntMeatProducts'] + 
    df['MntFishProducts'] + 
    df['MntSweetProducts'] + 
    df['MntGoldProds']
)
df['MntRegularProds'] = df['MntTotal'] - df['MntGoldProds'] # Criando a coluna com o valor gasto somente em produtos regulares

### Mesclando colunas com muitas categorias
- **AgeGroup:** Criando a coluna AgeGroup para agrupar as idades em intervalos, de forma que os ranges de idades fiquem bem definidos para serem utilizados futuramente nos modelos. 

In [9]:
df['AgeGroup'] = pd.cut( # Criando a coluna AgeGroup para agrupar as idades em intervalos de cerca de 15 anos
    df['Age'], # Definindo a coluna que será usada para criar os intervalos de idades
    bins=[18, 30, 45, 60, df['Age'].max()], # Definindo os intervalos das idades
    labels=['18-30', '31-45', '46-60', '61+'], # Definindo os rótulos de como os dados vão aparecer no dataset
    include_lowest=True # Incluindo 18 no intervalo
)

### Excluindo colunas que serviram de apoio
- **Dt_Customer:** Excluindo a coluna Dt_Customer para deixar o dataset apenas com informações relevantes.
- **Year_Birth:** Excluindo a coluna Year_Birth para deixar o dataset apenas com informações relevantes.
- **Age:** Excluindo a coluna Age para deixar o dataset apenas com informações relevantes.

In [10]:
df = df.drop([ # Excluindo as colunas que servirão de apoio
    'Dt_Customer', 
    'Year_Birth', 
    'Age', 
    'MntWines', 
    'MntFruits', 
    'MntMeatProducts', 
    'MntFishProducts', 
    'MntSweetProducts'
    ], 
    axis=1 # Definindo as colunas para serem excluídas
) 

### Visualizando a base tratada

In [11]:
df.head(3) # Exibindo as 3 primeiras linhas da base de dados

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Days_Since_Enrolled,Years_Since_Enrolled,MntTotal,MntRegularProds,AgeGroup
0,Graduation,Single,58138.0,0,0,58,88,3,8,10,...,0,0,0,0,1,663,1,1617,1529,46-60
1,Graduation,Single,46344.0,1,1,38,6,2,1,1,...,0,0,0,0,0,113,0,27,21,46-60
2,Graduation,Together,71613.0,0,0,26,42,1,8,2,...,0,0,0,0,0,312,0,776,734,46-60


In [12]:
df.tail(3) # Exibindo as 3 últimas linhas da base de dados

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,...,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Days_Since_Enrolled,Years_Since_Enrolled,MntTotal,MntRegularProds,AgeGroup
2237,Graduation,Divorced,56981.0,0,0,91,24,1,2,3,...,0,0,0,0,0,155,0,1241,1217,31-45
2238,Master,Together,69245.0,0,1,8,61,2,6,5,...,0,0,0,0,0,156,0,843,782,46-60
2239,PhD,Married,52869.0,1,1,40,21,3,3,1,...,0,0,0,0,1,622,1,172,151,46-60


In [13]:
df.info() # Exibindo as informações das variáveis da base de dados

<class 'pandas.core.frame.DataFrame'>
Index: 2216 entries, 0 to 2239
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Education             2216 non-null   object  
 1   Marital_Status        2216 non-null   object  
 2   Income                2216 non-null   float64 
 3   Kidhome               2216 non-null   int64   
 4   Teenhome              2216 non-null   int64   
 5   Recency               2216 non-null   int64   
 6   MntGoldProds          2216 non-null   int64   
 7   NumDealsPurchases     2216 non-null   int64   
 8   NumWebPurchases       2216 non-null   int64   
 9   NumCatalogPurchases   2216 non-null   int64   
 10  NumStorePurchases     2216 non-null   int64   
 11  NumWebVisitsMonth     2216 non-null   int64   
 12  AcceptedCmp3          2216 non-null   int64   
 13  AcceptedCmp4          2216 non-null   int64   
 14  AcceptedCmp5          2216 non-null   int64   
 15  AcceptedC

In [14]:
df.isnull().sum() # Somando todos os valores nulos de cada coluna

Education               0
Marital_Status          0
Income                  0
Kidhome                 0
Teenhome                0
Recency                 0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Response                0
Days_Since_Enrolled     0
Years_Since_Enrolled    0
MntTotal                0
MntRegularProds         0
AgeGroup                0
dtype: int64